In [1]:
import glob
from itertools import groupby
from pathlib import Path
import numpy as np
import pandas as pd
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.layers import Dense, Reshape
from keras.models import Model
from coremltools.converters.keras import convert
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
IMG_HEIGHT, IMG_WIDTH = (240, 240)
img_folder = Path("/home/soham/AeroMIT/dataset_exploiting/kmeans_version/").expanduser()

In [3]:
encoder_model = VGG16(input_shape=(IMG_HEIGHT,IMG_WIDTH,3), weights='imagenet', include_top=False)

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
np.prod(encoder_model.output.shape.as_list()[1:])

25088

In [5]:
len(encoder_model.get_weights())

26

In [6]:
import os
import fnmatch

In [7]:
image_filenames = []
for filename in os.listdir(img_folder):
    image_filenames.append(filename)

In [8]:
train_filenames, test_filenames = train_test_split(image_filenames, test_size=0.1)
print("test images: ", len(test_filenames))
print("train images", len(train_filenames))

test images:  7908
train images 71164


In [9]:
os.chdir(img_folder)
filename_order = []
def load_encode_images(encoder, filenames):
    batch_size = 16
    encoded_dim = np.prod(encoder.output.shape[1:]).value
    file_count = len(filenames)
    encoded = np.zeros((file_count, encoded_dim))
    for start_index in tqdm(list(range(0, file_count, batch_size))):
        end_index = min(start_index + batch_size, file_count)
        batch_filenames = filenames[start_index:end_index]

        batch_images = load_images(filename_order,batch_filenames)
        batch_encoded = encoder.predict(batch_images)
        batch_encoded_flat = batch_encoded.reshape(len(batch_images), -1)
        encoded[start_index:end_index, :] = batch_encoded_flat

    return encoded

def load_images(filename_order,filenames):
    images = np.zeros((len(filenames), IMG_HEIGHT, IMG_WIDTH, 3))
    for i, filename in enumerate(filenames):
        img = image.load_img(filename, target_size=(IMG_HEIGHT,IMG_WIDTH))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        images[i, :, :, :] = img_array
        filename_order.append([i,filename])
    return images

encoded_imgs = load_encode_images(encoder_model, train_filenames).T
print(filename_order)


[[0, 'sylvester01062.png'], [1, 'Other_Asada_100001669.png'], [2, 'Other_Asada_100001765.png'], [3, 'Kalal_Panda_01929.jpg'], [4, 'Kwon_VTD_skating2_frame_0549.jpg'], [5, 'Cehovin_gymnastics_100000566.png'], [6, 'Other_faceocc1_img1422.jpg'], [7, 'Kalal_Panda_02774.jpg'], [8, 'PROST_box_00493.jpg'], [9, 'Thang_person_person0121.png'], [10, 'Kwon_VTD_basketball_frame_0071.jpg'], [11, 'BoBot_Vid_K_cup_img0039.png'], [12, 'Other_drunk2_drunk0373.png'], [13, 'Kalal_Volkswagen_01366.jpg'], [14, 'Kwon_VTD_soccer_frame_0128.jpg'], [15, 'Wang_girl_mov_img-000092.png'], [0, 'Kwon_VTD_soccer_frame_0211.png'], [1, 'Kalal_CarChase_08975.jpg'], [2, 'Kwon_VTD_basketball_frame_0218.png'], [3, 'Kwon_VTD_singer1_frame_0023.png'], [4, 'Wang_girl_mov_img-000628.png'], [5, 'Kalal_CarChase_01587.jpg'], [6, 'Kalal_CarChase_05227.jpg'], [7, 'Kalal_Volkswagen_06751.jpg'], [8, 'Kalal_Volkswagen_04888.jpg'], [9, 'Ellis_ijcv2011_head_motion_input_000249.png'], [10, 'Kalal_car_UAVTest14169.jpg'], [11, 'Other_Asa

In [38]:
encoded_imgs = encoded_imgs.T
print(encoded_imgs.shape)


(71164, 25088)


In [41]:
num_steps = 50 # Total steps to train
batch_size = 1024 # The number of samples per batch
k = 20 # The number of clusters
num_features = 25088
#X = tf.placeholder(tf.float32, shape=[num_features,None])

In [28]:
from sklearn.cluster import MiniBatchKMeans

In [35]:
list1 = np.array([[1,2],[3,4]])

In [42]:
kmeans = MiniBatchKMeans(n_clusters=k, random_state=0, batch_size=200, max_iter=10).fit(encoded_imgs)

In [43]:
(kmeans.cluster_centers_).shape

(20, 25088)

In [44]:
labels = kmeans.labels_

In [45]:
np.savetxt("/home/soham/AeroMIT/rnn_yolo_model/keras-knn/labels_20_v2clusters.csv",labels,delimiter = ',')

In [46]:

with open("/home/soham/AeroMIT/rnn_yolo_model/keras-knn/image_filename_order.txt",'w') as f:
    for file in filename_order:
        f.write(str(file) + '\n')

In [59]:
labels_df = pd.DataFrame(labels)
labels_df['filenames'] = [file[1] for file in filename_order]
labels_df

,0,filenames
0,5,sylvester01062.png
1,7,Other_Asada_100001669.png
2,7,Other_Asada_100001765.png
3,4,Kalal_Panda_01929.jpg
4,7,Kwon_VTD_skating2_frame_0549.jpg
5,7,Cehovin_gymnastics_100000566.png
6,10,Other_faceocc1_img1422.jpg
7,10,Kalal_Panda_02774.jpg
8,1,PROST_box_00493.jpg
9,10,Thang_person_person0121.png


In [60]:
labels_df.to_csv('/home/soham/AeroMIT/rnn_yolo_model/keras-knn/labels_20_filenames.csv')

In [51]:
import gc
gc.collect()

616

In [52]:
os.chdir(img_folder)
prediction_filename_order = []
def load_predict_encode_images(encoder, filenames):
    batch_size = 16
    encoded_dim = np.prod(encoder.output.shape[1:]).value
    file_count = len(filenames)
    encoded = np.zeros((file_count, encoded_dim))
    for start_index in tqdm(list(range(0, file_count, batch_size))):
        end_index = min(start_index + batch_size, file_count)
        batch_filenames = filenames[start_index:end_index]

        batch_images = load_predict_images(filename_order,batch_filenames)
        batch_encoded = encoder.predict(batch_images)
        batch_encoded_flat = batch_encoded.reshape(len(batch_images), -1)
        encoded[start_index:end_index, :] = batch_encoded_flat

    return encoded

def load_predict_images(filename_order,filenames):
    images = np.zeros((len(filenames), IMG_HEIGHT, IMG_WIDTH, 3))
    for i, filename in enumerate(filenames):
        img = image.load_img(filename, target_size=(IMG_HEIGHT,IMG_WIDTH))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        images[i, :, :, :] = img_array
        prediction_filename_order.append([i,filename])
    return images

encoded_predict_imgs = load_predict_encode_images(encoder_model, test_filenames)
print(predict_filename_order)

NameError: name 'predict_filename_order' is not defined

In [53]:
print(prediction_filename_order)

[[0, 'Other_faceocc1_img1893.jpg'], [1, 'Kwon_VTD_football_frame_0231.jpg'], [2, 'Kalal_Volkswagen_05875.jpg'], [3, 'Other_drunk2_drunk0460.png'], [4, 'PROST_liquor_00721.jpg'], [5, 'Thang_tiger1_00000314.jpg'], [6, 'Ross_car11_00000120.png'], [7, 'Kalal_jumping_qianyi196.jpg'], [8, 'Kalal_Motocross_02216.png'], [9, 'Kalal_CarChase_02091.jpg'], [10, 'Kalal_Volkswagen_08116.jpg'], [11, 'Ross_dog1_dog100915.png'], [12, 'Kalal_Volkswagen_05699.jpg'], [13, 'Kalal_Panda_02052.jpg'], [14, 'Kalal_Volkswagen_06101.jpg'], [15, 'Kwon_VTD_basketball_frame_0532.jpg'], [0, 'Other_drunk2_drunk1504.jpg'], [1, 'Kwon_VTD_football_frame_0115.jpg'], [2, 'Kalal_pedestrian4_uavperson2043.jpg'], [3, 'Ross_dog1_dog100696.png'], [4, 'img0055.jpg'], [5, 'sylvester00934.png'], [6, 'Kwon_trans_img0094.jpg'], [7, 'Babenko_OccludedFace2_img00328.png'], [8, 'BoBot_Vid_K_cup_img0271.jpg'], [9, 'Kwon_VTD_skating1_frame_0307.jpg'], [10, 'PROST_liquor_00689.jpg'], [11, 'Ellis_ijcv2011_track_running_input_000354.png'], 

In [54]:
prediction_labels = kmeans.predict(encoded_predict_imgs)

In [61]:
predict_labels_df = pd.DataFrame(prediction_labels)


In [58]:
predict_labels_df

,0
0,5
1,7
2,7
3,4
4,7
5,7
6,10
7,10
8,1
9,10


In [62]:
predict_labels_df['filenames'] = [file[1] for file in prediction_filename_order]
predict_labels_df

,0,filenames
0,10,Other_faceocc1_img1893.jpg
1,7,Kwon_VTD_football_frame_0231.jpg
2,3,Kalal_Volkswagen_05875.jpg
3,10,Other_drunk2_drunk0460.png
4,15,PROST_liquor_00721.jpg
5,8,Thang_tiger1_00000314.jpg
6,10,Ross_car11_00000120.png
7,10,Kalal_jumping_qianyi196.jpg
8,10,Kalal_Motocross_02216.png
9,0,Kalal_CarChase_02091.jpg


In [63]:
predict_labels_df.to_csv('/home/soham/AeroMIT/rnn_yolo_model/keras-knn/predict_labels_20_filenames.csv')